# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "cis"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
ANKRD46,NP_001257308.1|NP_001257307.1,NP_001257308.1|NP_001257306.1,NP_001257308.1|NP_001257306.1,NP_001257307,8.594451e-02,NaN,0.941187,0.104310,0.168660,0.138009,0.520486,NaN,0.019340,0.411478,0.473017,0.467814
ASAP1,NP_060952.2|NP_001234925.1,NP_060952.2|NP_001234925.1|NP_001349854.1|NP_0...,NP_060952.2|NP_001234925.1|NP_001349854.1|NP_0...,NP_001234925,1.104415e-01,7.704115e-01,0.870043,0.732291,0.696628,0.189667,0.224021,0.038950,-0.012125,-0.070387,-0.083231,0.090577
ATAD2,NP_054828.2,NP_054828.2,NP_054828.2|NP_001341036.1|NP_060022.2|NP_0012...,NP_054828,2.807087e-08,2.136661e-01,0.274098,0.000031,0.001225,0.962834,1.690895,0.206485,0.140799,1.329348,1.055920,0.010878
ATP6V1C1,NP_001686.1,NP_001686.1,NP_001686.1,NP_001686,1.820055e-04,4.836484e-01,0.281444,0.545881,0.350692,0.000474,0.542337,0.040628,0.060945,-0.115075,0.197309,0.339252
C8orf37,NP_808880.1,NP_808880.1|NP_001350189.1,NP_808880.1|NP_001350189.1,NP_808880,1.618638e-01,NaN,0.789773,0.001189,0.055757,0.543773,0.330108,NaN,0.054388,0.776631,0.518921,0.126973
CA1,NP_001158302.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001278897.1,NP_001158302,1.281833e-01,5.182559e-01,0.995646,0.827523,0.816984,0.840179,0.856836,-0.162005,-0.001358,0.120970,-0.184345,-0.072096
CA1,NP_001158302.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001122301.1|NP_001278896.1,NP_001158302,1.281833e-01,5.182559e-01,0.995646,0.827523,0.817132,0.840179,0.856836,-0.162005,-0.001358,0.120970,-0.115336,-0.072096
CA1,NP_001158302.1|NP_001278896.1,NP_001278897.1,NP_001278897.1,NP_001158302,1.281833e-01,5.182559e-01,0.995646,0.879153,0.816984,0.840179,0.856836,-0.162005,-0.001358,0.115942,-0.184345,-0.072096
CA1,NP_001158302.1|NP_001278896.1,NP_001278897.1,NP_001122301.1|NP_001278896.1,NP_001158302,1.281833e-01,5.182559e-01,0.995646,0.879153,0.817132,0.840179,0.856836,-0.162005,-0.001358,0.115942,-0.115336,-0.072096


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,ANKRD46,NP_001257308.1|NP_001257307.1,8.594451e-02,0.520486
1,brca,ASAP1,NP_060952.2|NP_001234925.1,1.104415e-01,0.224021
2,brca,ATAD2,NP_054828.2,2.807087e-08,1.690895
3,brca,ATP6V1C1,NP_001686.1,1.820055e-04,0.542337
4,brca,C8orf37,NP_808880.1,1.618638e-01,0.330108
5,brca,CA13,NP_940986.1,1.532755e-01,0.486627
6,brca,CA3,NP_005172.1,2.085787e-01,0.700892
7,brca,CHMP4C,NP_689497.1,1.532755e-01,0.387163
8,brca,CNGB3,NP_061971.3,1.764023e-02,-0.824641
9,brca,COL14A1,NP_066933.1,1.028648e-04,-1.974448


## Select the proteins with a significant change, and take a detour to make some plots

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {TRANS_OR_CIS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [8]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,ATAD2,NP_054828.2,2.807087e-08,1.690895,1
1,brca,ATP6V1C1,NP_001686.1,1.820055e-04,0.542337,1
2,brca,CNGB3,NP_061971.3,1.764023e-02,-0.824641,-1
3,brca,COL14A1,NP_066933.1,1.028648e-04,-1.974448,-1
4,brca,COX6C,NP_004365.1,1.652870e-02,0.706711,1
5,brca,CPNE3,NP_003900.1|NP_055242.1|NP_705900.1,1.820055e-04,0.582023,1
6,brca,CSMD3,NP_937756.1|NP_937757.1|NP_443132.3,7.237739e-03,0.696771,1
7,brca,DCAF13,NP_056235.4,2.238654e-05,0.710523,1
8,brca,DECR1,NP_001317504.1|NP_001350.1,2.837961e-05,0.884562,1
9,brca,DPYS,NP_001376.1,1.047155e-02,-1.029501,-1


In [9]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
CPNE3,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
NUDCD1,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
OTUD6B,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
POP1,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
RIDA,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
RMDN1,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
STK3,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
YWHAZ,"[brca, colon, hnscc, lscc, luad, ovarian]",1,6
DCAF13,"[brca, colon, hnscc, lscc, luad]",1,5


In [10]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

6     8
5    10
4    10
3     9
2    13
1    28
Name: num_cancers, dtype: int64

## Save results

In [11]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")